# Hands-On: Feature Engineering

This hands-on will cover some techniques in feature engineering in combination with data cleaning and processing.

It covers feature extraction from `datetime` objects, feature transformation from existing variables in the dataset, feature generation and some categorical encoding such as count, label and one-hot encoding.

In [1]:
import os
import math
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder, LabelEncoder

%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Data - CitiBike Trip Histories

CitiBike provides the data of the bike share through this website: https://www.citibikenyc.com/system-data

For this exercise, we'll be using their trip history data which may be found [here](https://s3.amazonaws.com/tripdata/index.html). 

Kindly choose a 2021 month file to download (not the ones with JC as the prefix). You can download data for January or Febuary (20-50MB in file size for slow internet connections) and extract it to your data folder.

**CitiBike trip data includes:**

* Trip Duration (seconds)
* Start Time and Date
* Stop Time and Date
* Start Station Name
* End Station Name
* Station ID
* Station Lat/Long
* Bike ID
* User Type (Customer = 24-hour pass or 3-day pass user; Subscriber = Annual Member)
* Gender (Zero=unknown; 1=male; 2=female)
* Year of Birth

In [2]:
data = pd.read_csv(os.getenv('DSDATA') + '/202102-citibike-tripdata.csv')
data.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,304,2021-02-01 00:04:23.0780,2021-02-01 00:09:27.7920,3175,W 70 St & Amsterdam Ave,40.777480,-73.982886,4045,West End Ave & W 60 St,40.772370,-73.990050,27451,Subscriber,1996,2
1,370,2021-02-01 00:07:08.8080,2021-02-01 00:13:19.4670,3154,E 77 St & 3 Ave,40.773142,-73.958562,3725,2 Ave & E 72 St,40.768762,-73.958408,35000,Subscriber,1991,1
2,635,2021-02-01 00:07:55.9390,2021-02-01 00:18:31.0390,502,Henry St & Grand St,40.714211,-73.981095,411,E 6 St & Avenue D,40.722281,-73.976687,49319,Subscriber,1980,2
3,758,2021-02-01 00:08:42.0960,2021-02-01 00:21:20.7820,3136,5 Ave & E 63 St,40.766368,-73.971518,3284,E 88 St & Park Ave,40.781411,-73.955959,48091,Customer,1969,0
4,522,2021-02-01 00:09:32.6820,2021-02-01 00:18:15.4100,505,6 Ave & W 33 St,40.749013,-73.988484,3687,E 33 St & 1 Ave,40.743227,-73.974498,48596,Subscriber,1988,1


In [3]:
data.dtypes

tripduration                 int64
starttime                   object
stoptime                    object
start station id             int64
start station name          object
start station latitude     float64
start station longitude    float64
end station id               int64
end station name            object
end station latitude       float64
end station longitude      float64
bikeid                       int64
usertype                    object
birth year                   int64
gender                       int64
dtype: object

### Data Types

These are the types of the dataset. We have a mix of **categorical** (Stations, User Type, Gender), **temporal** (Start Time, Stop Time, Trip Duration) **spatial** and **numerical** data.

## Feature Extraction

From the lecture, we mentioned that we can extract additional information from `datetime` features. Prior to extracting these features, we need to ensure that the `datetime` feature is a `datetime` object in Python so we can easily use the functions available to extract the information.

However, sometimes date and time columns are not automatically converted into a `datetime` object, they are loaded as a `str`. We can be convert the column in two ways:

1. During `pd.read_csv` using the `parse_dates` argument and passing the column name or the index of the column.
2. `pd.to_datetime` method by updating the column.

In [4]:
data['starttime'] = pd.to_datetime(data['starttime'])
data['stoptime'] = pd.to_datetime(data['stoptime'])

In [5]:
data.dtypes

tripduration                        int64
starttime                  datetime64[ns]
stoptime                   datetime64[ns]
start station id                    int64
start station name                 object
start station latitude            float64
start station longitude           float64
end station id                      int64
end station name                   object
end station latitude              float64
end station longitude             float64
bikeid                              int64
usertype                           object
birth year                          int64
gender                              int64
dtype: object

#### Datetime-like properties in `pandas`

Datetimelike properties in a `pandas.Series` can be accessed through `Series.dt`

See the [`pandas` documentation for datetimelike properties](https://pandas.pydata.org/pandas-docs/stable/reference/series.html#datetimelike-properties) to see all possible values.

For this exercise, let's extract the **day of the week** and the **hour of the day** from the starttime. Create a new column for these values.

In [29]:
data['dayofweek'] = data['starttime'].dt.dayofweek
data['hourofday'] = data['starttime'].dt.hour
data['year'] = data['starttime'].dt.year

In [30]:
data.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,...,bikeid,usertype,birth year,gender,dayofweek,hourofday,duration_min,age,distance_km,year
0,304,2021-02-01 00:04:23.078,2021-02-01 00:09:27.792,3175,W 70 St & Amsterdam Ave,40.777480,-73.982886,4045,West End Ave & W 60 St,40.772370,...,27451,Subscriber,1996,2,0,0,5.066667,25,0.828754,2021
1,370,2021-02-01 00:07:08.808,2021-02-01 00:13:19.467,3154,E 77 St & 3 Ave,40.773142,-73.958562,3725,2 Ave & E 72 St,40.768762,...,35000,Subscriber,1991,1,0,0,6.166667,30,0.487242,2021
2,635,2021-02-01 00:07:55.939,2021-02-01 00:18:31.039,502,Henry St & Grand St,40.714211,-73.981095,411,E 6 St & Avenue D,40.722281,...,49319,Subscriber,1980,2,0,0,10.583333,41,0.971186,2021
3,758,2021-02-01 00:08:42.096,2021-02-01 00:21:20.782,3136,5 Ave & E 63 St,40.766368,-73.971518,3284,E 88 St & Park Ave,40.781411,...,48091,Customer,1969,0,0,0,12.633333,52,2.124707,2021
4,522,2021-02-01 00:09:32.682,2021-02-01 00:18:15.410,505,6 Ave & W 33 St,40.749013,-73.988484,3687,E 33 St & 1 Ave,40.743227,...,48596,Subscriber,1988,1,0,0,8.700000,33,1.342431,2021


## Feature Transformation

Currently, the trip duration is in seconds. Depending on the use case or analysis, using seconds might not be easily interpreted by most since we're used to either a trip lasting minutes or hours. 

Let's transform the tripduration in seconds to minutes and see how long the trip actually took.

In [31]:
data['duration_min'] = data['tripduration']/60
data.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,...,bikeid,usertype,birth year,gender,dayofweek,hourofday,duration_min,age,distance_km,year
0,304,2021-02-01 00:04:23.078,2021-02-01 00:09:27.792,3175,W 70 St & Amsterdam Ave,40.777480,-73.982886,4045,West End Ave & W 60 St,40.772370,...,27451,Subscriber,1996,2,0,0,5.066667,25,0.828754,2021
1,370,2021-02-01 00:07:08.808,2021-02-01 00:13:19.467,3154,E 77 St & 3 Ave,40.773142,-73.958562,3725,2 Ave & E 72 St,40.768762,...,35000,Subscriber,1991,1,0,0,6.166667,30,0.487242,2021
2,635,2021-02-01 00:07:55.939,2021-02-01 00:18:31.039,502,Henry St & Grand St,40.714211,-73.981095,411,E 6 St & Avenue D,40.722281,...,49319,Subscriber,1980,2,0,0,10.583333,41,0.971186,2021
3,758,2021-02-01 00:08:42.096,2021-02-01 00:21:20.782,3136,5 Ave & E 63 St,40.766368,-73.971518,3284,E 88 St & Park Ave,40.781411,...,48091,Customer,1969,0,0,0,12.633333,52,2.124707,2021
4,522,2021-02-01 00:09:32.682,2021-02-01 00:18:15.410,505,6 Ave & W 33 St,40.749013,-73.988484,3687,E 33 St & 1 Ave,40.743227,...,48596,Subscriber,1988,1,0,0,8.700000,33,1.342431,2021


## Feature Generation

Calculate the age from the birth year.

Observe the min and max of the calculated birth year. Notice anything problematic?

In [9]:
data['age'] = data['starttime'].dt.year - data['birth year']
data.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,dayofweek,hourofday,duration_min,age
0,304,2021-02-01 00:04:23.078,2021-02-01 00:09:27.792,3175,W 70 St & Amsterdam Ave,40.777480,-73.982886,4045,West End Ave & W 60 St,40.772370,-73.990050,27451,Subscriber,1996,2,0,0,5.066667,25
1,370,2021-02-01 00:07:08.808,2021-02-01 00:13:19.467,3154,E 77 St & 3 Ave,40.773142,-73.958562,3725,2 Ave & E 72 St,40.768762,-73.958408,35000,Subscriber,1991,1,0,0,6.166667,30
2,635,2021-02-01 00:07:55.939,2021-02-01 00:18:31.039,502,Henry St & Grand St,40.714211,-73.981095,411,E 6 St & Avenue D,40.722281,-73.976687,49319,Subscriber,1980,2,0,0,10.583333,41
3,758,2021-02-01 00:08:42.096,2021-02-01 00:21:20.782,3136,5 Ave & E 63 St,40.766368,-73.971518,3284,E 88 St & Park Ave,40.781411,-73.955959,48091,Customer,1969,0,0,0,12.633333,52
4,522,2021-02-01 00:09:32.682,2021-02-01 00:18:15.410,505,6 Ave & W 33 St,40.749013,-73.988484,3687,E 33 St & 1 Ave,40.743227,-73.974498,48596,Subscriber,1988,1,0,0,8.700000,33


In [10]:
data['age'].describe()

count    634631.000000
mean         49.254447
std           8.307802
min          17.000000
25%          52.000000
50%          52.000000
75%          52.000000
max         136.000000
Name: age, dtype: float64

#### Distance

Another feature we can generate from the data is distance. Although the provided values are in longitude and latitudes and they're measured in degrees, the distance calculated from these points would also be in degrees (and not meters). 

There's actually a library that specifically handles geospatial data called `geopy` ([Link](https://geopy.readthedocs.io/en/stable/#module-geopy.distance)). For simplicity sake in this tutorial, we use an existing function that calculates the geodesic distance using the Haversine formula given the starting and ending longitude and latitudes: `calculate_distance(lat1, lon1, lat2, lon2)`

Credits to [Wayne Dyck](https://gist.github.com/rochacbruno/2883505) for the function.

In [11]:
def calculate_distance(lat1, lon1, lat2, lon2):
    """
    Calculates the distance provided a pair of longitudes and latitudes
    using the Haversine formula
    
    Returns the distance in kilometers.
    """
    radius = 6371 # km

    dlat = math.radians(lat2-lat1)
    dlon = math.radians(lon2-lon1)
    a = math.sin(dlat/2) * math.sin(dlat/2) + math.cos(math.radians(lat1)) \
        * math.cos(math.radians(lat2)) * math.sin(dlon/2) * math.sin(dlon/2)
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    d = radius * c

    return d

In [12]:
data['distance_km'] = data.apply(lambda x: calculate_distance(x['start station latitude'], x['start station longitude'],
                                        x['end station latitude'], x['end station longitude']), axis=1)

In [13]:
data.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender,dayofweek,hourofday,duration_min,age,distance_km
0,304,2021-02-01 00:04:23.078,2021-02-01 00:09:27.792,3175,W 70 St & Amsterdam Ave,40.777480,-73.982886,4045,West End Ave & W 60 St,40.772370,-73.990050,27451,Subscriber,1996,2,0,0,5.066667,25,0.828754
1,370,2021-02-01 00:07:08.808,2021-02-01 00:13:19.467,3154,E 77 St & 3 Ave,40.773142,-73.958562,3725,2 Ave & E 72 St,40.768762,-73.958408,35000,Subscriber,1991,1,0,0,6.166667,30,0.487242
2,635,2021-02-01 00:07:55.939,2021-02-01 00:18:31.039,502,Henry St & Grand St,40.714211,-73.981095,411,E 6 St & Avenue D,40.722281,-73.976687,49319,Subscriber,1980,2,0,0,10.583333,41,0.971186
3,758,2021-02-01 00:08:42.096,2021-02-01 00:21:20.782,3136,5 Ave & E 63 St,40.766368,-73.971518,3284,E 88 St & Park Ave,40.781411,-73.955959,48091,Customer,1969,0,0,0,12.633333,52,2.124707
4,522,2021-02-01 00:09:32.682,2021-02-01 00:18:15.410,505,6 Ave & W 33 St,40.749013,-73.988484,3687,E 33 St & 1 Ave,40.743227,-73.974498,48596,Subscriber,1988,1,0,0,8.700000,33,1.342431


#### Count Encoding

Using the trips raw can be quite useful, but what if we had a different problem we wanted to solve. For example, we want to use the number of trips per origin-destination pair to plan for the initial placement of bikes per station so we can balance out the supply and demand.

We would simply need the number of trips (or bikes used) per origin-destination.

In [15]:
## Count of rides per OD
od_trips = data.groupby(['start station name', 'end station name'], as_index=False)['bikeid'].count()
od_trips.head()

,start station name,end station name,bikeid
0,1 Ave & E 110 St,1 Ave & E 110 St,17
1,1 Ave & E 110 St,1 Ave & E 68 St,11
2,1 Ave & E 110 St,1 Ave & E 78 St,1
3,1 Ave & E 110 St,1 Ave & E 94 St,2
4,1 Ave & E 110 St,2 Ave & E 104 St,2


Alternatively, we can also make this into an origin-destination matrix - which can be useful for visualization and also comparison.

In [16]:
pd.pivot_table(od_trips, index='start station name', columns='end station name', values='bikeid')

end station name,1 Ave & E 110 St,1 Ave & E 16 St,1 Ave & E 18 St,1 Ave & E 30 St,1 Ave & E 39 St,1 Ave & E 44 St,1 Ave & E 5 St,1 Ave & E 62 St,1 Ave & E 68 St,1 Ave & E 78 St,...,Wilson Ave & Troutman St,Windsor Pl & 8 Ave,Withers St & Kingsland Ave,Wolcott St & Dwight St,Woodward Ave & Harman St,Wyckoff Av & Jefferson St,Wyckoff Av & Stanhope St,Wyckoff St & Nevins St,Wythe Ave & Metropolitan Ave,York St
start station name,,,,,,,,,,,,,,,,,,,,,
1 Ave & E 110 St,17.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1 Ave & E 16 St,1.0,31.0,78.0,133.0,32.0,8.0,12.0,13.0,9.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1 Ave & E 18 St,NaN,66.0,57.0,58.0,40.0,NaN,4.0,3.0,6.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1 Ave & E 30 St,3.0,86.0,14.0,22.0,60.0,25.0,3.0,10.0,16.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1.0,NaN
1 Ave & E 39 St,1.0,13.0,18.0,7.0,68.0,54.0,4.0,37.0,38.0,34.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Woodward Ave & Harman St,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3.0,NaN,10.0,NaN,NaN,NaN
Wyckoff Av & Jefferson St,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,1.0,4.0,1.0,NaN,NaN,NaN
Wyckoff Av & Stanhope St,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,NaN,NaN,NaN,5.0,1.0,4.0,NaN,1.0,NaN


#### Label Encoding and One-Hot Encoding

`scikit-learn` provides encoding functions for preprocessing of data before model training. In this example, we'll use the `usertype` and `gender` categories of the Citibike data for illustration on how to use these encoders.

For further reading on other preprocessing techiniques scikit learn offers, see this [article](https://scikit-learn.org/stable/modules/preprocessing.html).

In [17]:
users = data[['bikeid', 'usertype', 'gender']]
users.head()

,bikeid,usertype,gender
0,27451,Subscriber,2
1,35000,Subscriber,1
2,49319,Subscriber,2
3,48091,Customer,0
4,48596,Subscriber,1


In [18]:
le_usertype = LabelEncoder()
users['user_encoded'] = le_usertype.fit_transform(users.usertype)

users.head()

/Users/brianesamson/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,bikeid,usertype,gender,user_encoded
0,27451,Subscriber,2,1
1,35000,Subscriber,1,1
2,49319,Subscriber,2,1
3,48091,Customer,0,0
4,48596,Subscriber,1,1


In [19]:
gender_ohe = OneHotEncoder()
usertype_ohe = OneHotEncoder()
X_gender = gender_ohe.fit_transform(users.gender.values.reshape(-1,1)).toarray()
X_usertype = usertype_ohe.fit_transform(users.usertype.values.reshape(-1,1)).toarray()

In [20]:
X_gender

array([[0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [21]:
X_usertype

array([[0., 1.],
       [0., 1.],
       [0., 1.],
       ...,
       [1., 0.],
       [0., 1.],
       [1., 0.]])

In [22]:
users_OH = pd.DataFrame(X_gender, columns = ["gender_"+str(int(i)) for i in range(X_gender.shape[1])])
users_test = pd.concat([users, users_OH], axis=1)

users_OH = pd.DataFrame(X_usertype, columns = ["usertype_"+str(int(i)) for i in range(X_usertype.shape[1])])
users_test = pd.concat([users_test, users_OH], axis=1)

In [23]:
users_test.head()

,bikeid,usertype,gender,user_encoded,gender_0,gender_1,gender_2,usertype_0,usertype_1
0,27451,Subscriber,2,1,0.0,0.0,1.0,0.0,1.0
1,35000,Subscriber,1,1,0.0,1.0,0.0,0.0,1.0
2,49319,Subscriber,2,1,0.0,0.0,1.0,0.0,1.0
3,48091,Customer,0,0,1.0,0.0,0.0,1.0,0.0
4,48596,Subscriber,1,1,0.0,1.0,0.0,0.0,1.0


#### Label Encoding (using only `pandas`!!!)

`pandas` also has a nifty feature of turning categorical values into numerical labels. 

Steps:
1. Convert the type of the column into category
2. Use `cat.codes` to get the integer label for each value

In [24]:
users.head()

,bikeid,usertype,gender,user_encoded
0,27451,Subscriber,2,1
1,35000,Subscriber,1,1
2,49319,Subscriber,2,1
3,48091,Customer,0,0
4,48596,Subscriber,1,1


In [25]:
users['usertype'] = users['usertype'].astype('category')
users.dtypes

/Users/brianesamson/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


bikeid             int64
usertype        category
gender             int64
user_encoded       int64
dtype: object

In [26]:
users['usertype'].head()

0    Subscriber
1    Subscriber
2    Subscriber
3      Customer
4    Subscriber
Name: usertype, dtype: category
Categories (2, object): ['Customer', 'Subscriber']

In [27]:
users['usertype_code'] = users['usertype'].cat.codes
users.head()

/Users/brianesamson/.pyenv/versions/3.7.6/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,bikeid,usertype,gender,user_encoded,usertype_code
0,27451,Subscriber,2,1,1
1,35000,Subscriber,1,1,1
2,49319,Subscriber,2,1,1
3,48091,Customer,0,0,0
4,48596,Subscriber,1,1,1


### One Hot Encoding using `pandas`

`pandas` also has this function called `get_dummies` where it will provide you the binary flags for each category in a column.

In [28]:
pd.get_dummies(users, columns=['usertype']).head()

,bikeid,gender,user_encoded,usertype_code,usertype_Customer,usertype_Subscriber
0,27451,2,1,1,0,1
1,35000,1,1,1,0,1
2,49319,2,1,1,0,1
3,48091,0,0,0,1,0
4,48596,1,1,1,0,1
